In [3]:
!pip install transformers -U

In [4]:
import pandas as pd
from datasets import Dataset
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel

ImportError: accelerate>=0.20.3 is required for a normal functioning of this module, but found accelerate==0.5.1.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git main

In [ ]:
train_df = pd.read_csv("../data/train.csv")
valid_df = pd.read_csv("../data/valid.csv")
train_df.head()

In [ ]:
train_df["label"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

In [ ]:
train_df["Words Per Review"] = train_df["text"].str.split().apply(len)
train_df.boxplot("Words Per Review", by="label", grid=False, showfliers=False,
           color="gray")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
train_df.drop(['Words Per Review'], axis=1, inplace=True)

In [ ]:
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
train_ds,valid_ds

In [ ]:
model_ckpt = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
encoded_text = tokenizer(train_df["text"][0])
encoded_text

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
train_encoded = train_ds.map(tokenize, batched=True, batch_size=1024)

In [ ]:
valid_encoded = valid_ds.map(tokenize, batched=True, batch_size=1024)

In [ ]:
train_encoded

In [ ]:
device = "mps"
model = AutoModel.from_pretrained(model_ckpt).to(device)

In [ ]:
def extract_hidden_states(batch):
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    return {"hidden_state": last_hidden_state[:,0].cpu().object()}

In [ ]:
train_encoded.set_format("torch",
                        columns=["input_ids", "attention_mask", "label"])
valid_encoded.set_format("torch",
                        columns=["input_ids", "attention_mask", "label"])

In [21]:
train_hidden = train_encoded.map(extract_hidden_states, batched=True)

  0%|          | 0/460 [00:00<?, ?ba/s]

AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations